**Status**: Still needs work.

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [30]:
df = pd.read_csv('House_Rent_Dataset.csv',
                parse_dates = ['Posted On'])
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [31]:
df['posted_month'] = df['Posted On'].dt.month
df['posted_day'] = df['Posted On'].dt.day
df['posted_day_week'] = df['Posted On'].dt.dayofweek

In [32]:
df.drop('Posted On', axis = 1, inplace = True)

In [33]:
from sklearn.preprocessing import OrdinalEncoder

In [34]:
first = []
second = []
floor_split_series = df.Floor.str.split(" out of ")

for index, value in floor_split_series.items():
    if len(value) < 2:
        first.append(value[0])
        second.append(value[0])
    elif 'Basement' in str(value[0]):
        first.append(402)
        second.append(str(value[1]))
    elif 'Ground' in str(value[0]):
        first.append(401)
        second.append(str(value[1]))
    else:
        first.append(str(value[0]))
        second.append(str(value[1]))
        
first = pd.Series(first)
second = pd.Series(second)
floor = pd.concat([first, second], axis = 1)

floor.iloc[2883] = [401,401]

floor = floor.astype('int')
floor = floor.add(2)
floor.replace({403: 1, 404: 0}, inplace = True)

floor.rename(columns = {0: 'floor_received', 1: 'floor_out_of'}, inplace = True)
df = pd.concat([df, floor], axis = 1)

df.drop('Floor', axis = 1, inplace = True)

In [35]:
cols_obj = list(df.select_dtypes(include = 'object').columns)
cols_obj

['Area Type',
 'Area Locality',
 'City',
 'Furnishing Status',
 'Tenant Preferred',
 'Point of Contact']

In [36]:
df.loc[:, cols_obj].nunique()

Area Type               3
Area Locality        2235
City                    6
Furnishing Status       3
Tenant Preferred        3
Point of Contact        3
dtype: int64

In [37]:
for col in cols_obj:
    display(df.loc[:, col].unique())

array(['Super Area', 'Carpet Area', 'Built Area'], dtype=object)

array(['Bandel', 'Phool Bagan, Kankurgachi', 'Salt Lake City Sector 2',
       ..., 'BN Reddy Nagar', 'Godavari Homes, Quthbullapur',
       'Manikonda, Hyderabad'], dtype=object)

array(['Kolkata', 'Mumbai', 'Bangalore', 'Delhi', 'Chennai', 'Hyderabad'],
      dtype=object)

array(['Unfurnished', 'Semi-Furnished', 'Furnished'], dtype=object)

array(['Bachelors/Family', 'Bachelors', 'Family'], dtype=object)

array(['Contact Owner', 'Contact Agent', 'Contact Builder'], dtype=object)

# Separating X and y

In [38]:
X = df.drop('Rent', axis = 1)
y = df.loc[:, 'Rent']

### Ordinal Variables:
* Area Type
* Furnishing Status

### Nominal Variables:
* Tenant Preferred
* Point of Contact
* City
* Area Locality

In [25]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

### Area Type

In [76]:
oe_area = OrdinalEncoder(categories = [['','Carpet Area', 'Built Area', 'Super Area']])

In [77]:
area_arr = df.loc[:, 'Area Type'].values.reshape(-1,1)
area_arr.shape

(4746, 1)

In [78]:
area_arr = oe_area.fit_transform(area_arr)

### Furnishing Status

In [79]:
oe_fs = OrdinalEncoder(categories = [['','Unfurnished','Semi-Furnished','Furnished']])

In [80]:
fs_arr = X.loc[:, 'Furnishing Status'].values.reshape(-1,1)

In [81]:
fs_arr = oe_fs.fit_transform(fs_arr)

Adding these encoded columns to the main X

In [94]:
fs_arr_df = pd.DataFrame(fs_arr).rename(columns = {0:'furnishing_status_encoded'})
at_arr_df = pd.DataFrame(area_arr).rename(columns = {0: 'area_type_encoded'})

In [96]:
X = pd.concat([X, fs_arr_df, at_arr_df], axis = 1).drop(['Area Type', 'Furnishing Status'], axis = 1)
X.head()

,BHK,Size,Area Locality,City,Tenant Preferred,Bathroom,Point of Contact,posted_month,posted_day,posted_day_week,floor_received,floor_out_of,furnishing_status_encoded,area_type_encoded
0,2,1100,Bandel,Kolkata,Bachelors/Family,2,Contact Owner,5,18,2,1,4,1.00,3.00
1,2,800,"Phool Bagan, Kankurgachi",Kolkata,Bachelors/Family,1,Contact Owner,5,13,4,3,5,2.00,3.00
2,2,1000,Salt Lake City Sector 2,Kolkata,Bachelors/Family,1,Contact Owner,5,16,0,3,5,2.00,3.00
3,2,800,Dumdum Park,Kolkata,Bachelors/Family,1,Contact Owner,7,4,0,3,4,1.00,3.00
4,2,850,South Dum Dum,Kolkata,Bachelors,1,Contact Owner,5,9,0,3,4,1.00,1.00


## Nominal Categorical Columns

### Nominal Variables:
* Tenant Preferred
* Point of Contact
* City
* Area Locality

In [101]:
nom_obj_cols = ['Tenant Preferred', 'Point of Contact', 'City', 'Area Locality']

In [102]:
nom_obj_cols

['Tenant Preferred', 'Point of Contact', 'City', 'Area Locality']

In [103]:
df.loc[:, nom_obj_cols].nunique()

Tenant Preferred       3
Point of Contact       3
City                   6
Area Locality       2235
dtype: int64

## Tenant Preferred

In [120]:
X = pd.concat([X, pd.get_dummies(X.loc[:, 'Tenant Preferred'], prefix = 'tenant', drop_first=True)], axis = 1).drop('Tenant Preferred', axis = 1)

## Point of Contact

In [123]:
X = pd.concat([X, pd.get_dummies(X.loc[:, 'Point of Contact'], prefix = 'contact', drop_first=True)], axis = 1).drop('Point of Contact', axis = 1)

## City

In [124]:
X = pd.concat([X, pd.get_dummies(X.loc[:, 'City'], prefix = 'city', drop_first=True)], axis = 1).drop('City', axis = 1)

## Area Locality

In [125]:
X_no_area = X.copy()

In [126]:
X = pd.concat([X, pd.get_dummies(X.loc[:, 'Area Locality'], prefix = 'area', drop_first=True)], axis = 1).drop('Area Locality', axis = 1)

In [127]:
X.head()

,BHK,Size,Bathroom,posted_month,posted_day,posted_day_week,floor_received,floor_out_of,furnishing_status_encoded,area_type_encoded,...,area_sra,area_sri sai arcade madinaguda,area_sspdl Mayfair,area_sunanda circal,area_tollygunge Kabardanga,"area_vanamali chs ghatla, Ghatla",area_venkatapuram,area_venkatesa perumal nagar,area_villvam towers tnhb colony,area_whitefield
0,2,1100,2,5,18,2,1,4,1.00,3.00,...,0,0,0,0,0,0,0,0,0,0
1,2,800,1,5,13,4,3,5,2.00,3.00,...,0,0,0,0,0,0,0,0,0,0
2,2,1000,1,5,16,0,3,5,2.00,3.00,...,0,0,0,0,0,0,0,0,0,0
3,2,800,1,7,4,0,3,4,1.00,3.00,...,0,0,0,0,0,0,0,0,0,0
4,2,850,1,5,9,0,3,4,1.00,1.00,...,0,0,0,0,0,0,0,0,0,0


---------------------------------------

# Preprocessing before fitting
* Normalizing the label
* Scaling the features

In [128]:
from scipy.stats.mstats import normaltest
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from scipy.stats import uniform

In [129]:
y_norm, lam = boxcox(y)

In [137]:
from sklearn.model_selection import train_test_split

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y_norm, test_size = 0.2)

In [139]:
from sklearn.preprocessing import StandardScaler

In [140]:
ss = StandardScaler()

In [141]:
ss = ss.fit(X_train)
X_train_s = ss.transform(X_train)
X_test_s = ss.transform(X_test)

In [142]:
y_train_inv = inv_boxcox(y_train, lam)
y_test_inv = inv_boxcox(y_test, lam)

In [152]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_squared_log_error
def scores(y, pred):
    print('Mean Absolute Error', mean_absolute_error(y, pred))
    print("Root Mean Squared Error", np.sqrt(mean_squared_error(y, pred)))
    print("Mean Squared Log Error", mean_squared_log_error(y, pred))
    print("R2 Score", r2_score(y, pred))

## Trying out the default XGBoost regressor

In [143]:
from xgboost import XGBRegressor

In [146]:
model = XGBRegressor()
xgbr_default = model.fit(X_train_s, y_train)

In [150]:
xgbr_default_pred = xgbr_default.predict(X_test_s)
xgbr_default_pred_inv = inv_boxcox(xgbr_default_pred, lam)

In [153]:
scores(y_test_inv, xgbr_default_pred_inv)

Mean Absolute Error 10306.062894479901
Root Mean Squared Error 31311.011097057268
Mean Squared Log Error 0.1359342849748245
R2 Score 0.7285294900841661


In [157]:
xg_fi = pd.Series(index = X.columns, data = xgbr_default.feature_importances_)
xg_fi.nlargest(10)

city_Mumbai             0.13
Bathroom                0.12
contact_Contact Owner   0.09
city_Kolkata            0.02
BHK                     0.01
area_Ram Nagar          0.01
area_Shyam Bazar        0.01
city_Delhi              0.01
area_Uppal, NH 2 2      0.01
area_Laxmi Nagar        0.01
dtype: float32

# Trying out different learners

In [182]:
def model_tester(model, X_train, X_test, y_train, y_test, model_name):
    model = model.fit(X_train, y_train)
    pred_norm = model.predict(X_test)
    
    pred_inv = inv_boxcox(pred_norm, lam)
    
    mae = mean_absolute_error(y_test_inv, pred_inv)
    mse = mean_squared_error(y_test_inv, pred_inv)
    msle = mean_squared_log_error(y_test_inv, pred_inv)
    r2 = r2_score(y_test_inv, pred_inv)
    rmse = np.sqrt(mse)
    
    performance = pd.DataFrame(index = [model_name],
                              data = {'MAE': mae,
                                     'MSE': mse,
                                     'MSLE': msle,
                                     'R2': r2,
                                     'RMSE': rmse})
    
    return [model, performance]

In [183]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [184]:
scores_list = []

In [185]:
model = SVR()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'SVR'))

In [186]:
model = CatBoostRegressor()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'CatBoostRegressor'))

Learning rate set to 0.050549
0:	learn: 0.0283149	total: 12.1ms	remaining: 12.1s
1:	learn: 0.0274037	total: 20.3ms	remaining: 10.1s
2:	learn: 0.0265603	total: 30.1ms	remaining: 10s
3:	learn: 0.0257308	total: 37.6ms	remaining: 9.37s
4:	learn: 0.0249512	total: 45.2ms	remaining: 8.99s
5:	learn: 0.0242185	total: 52.2ms	remaining: 8.65s
6:	learn: 0.0235332	total: 59.4ms	remaining: 8.43s
7:	learn: 0.0228948	total: 66.5ms	remaining: 8.24s
8:	learn: 0.0223288	total: 73.3ms	remaining: 8.07s
9:	learn: 0.0217931	total: 80.4ms	remaining: 7.96s
10:	learn: 0.0212744	total: 87.4ms	remaining: 7.86s
11:	learn: 0.0207652	total: 94.8ms	remaining: 7.8s
12:	learn: 0.0203314	total: 102ms	remaining: 7.73s
13:	learn: 0.0198770	total: 110ms	remaining: 7.71s
14:	learn: 0.0195008	total: 117ms	remaining: 7.67s
15:	learn: 0.0191079	total: 124ms	remaining: 7.65s
16:	learn: 0.0187310	total: 131ms	remaining: 7.6s
17:	learn: 0.0184137	total: 139ms	remaining: 7.57s
18:	learn: 0.0180943	total: 146ms	remaining: 7.53s
19:

In [187]:
model = LGBMRegressor()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'LGBMR'))

In [188]:
model = RandomForestRegressor()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'RandomForest'))

In [189]:
model = ExtraTreesRegressor()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'ExtraTreesRegressor'))

In [190]:
model = LassoCV()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'Lasso_cv'))

In [191]:
model = RidgeCV()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'Ridge_cv'))

In [192]:
model = XGBRegressor()
scores_list.append(model_tester(model, X_train_s, X_test_s, y_train, y_test, 'XGBoostRegressor'))

In [195]:
pd.concat([i[1] for i in scores_list], axis = 0).sort_values('R2', ascending = False)

,MAE,MSE,MSLE,R2,RMSE
RandomForest,9749.49,869876679.25,0.14,0.76,29493.67
ExtraTreesRegressor,9974.94,901138239.68,0.15,0.75,30018.96
XGBoostRegressor,10306.06,980379415.92,0.14,0.73,31311.01
LGBMR,10525.52,1030158658.60,0.14,0.71,32096.08
CatBoostRegressor,10554.56,1068163910.57,0.14,0.70,32682.78
SVR,26252.57,4263312290.37,1.60,-0.18,65294.04
Ridge_cv,78185.62,3018146901914.94,0.23,-834.74,1737281.47
Lasso_cv,976833.65,671467455044934.38,0.28,-185930.70,25912689.07


# Observations:
XGBoost, RandomForest and ExtraTreesRegressor are working best

## XGBoost Regressor

In [ ]:
param_grid_xgbr = {'gamma':        [4,12.8,25.6,51.2,102.4, 200],
                  'learning_rate': [0.300000012, 0.4, 0.5, 0.6, 0.7],
                  'max_depth':     [12,13,14],
                  'n_estimators':  [50,65,80,100,115,130,150],
                  'reg_alpha':     [12.8,25.6,51.2,102.4,200],
                  'reg_lambda':    [,12.8,25.6,51.2,102.4,200],
                  'subsample':     uniform(0.6, 0.4)}

In [204]:
param_grid_xgbr_low_lr = {'gamma':        [4,12.8,25.6,51.2,102.4, 200],
                  'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25],
                  'max_depth':     [12,13,14],
                  'n_estimators':  [50,65,80,100,115,130,150],
                  'reg_alpha':     [12.8,25.6],
                  'reg_lambda':    [12.8,25.6,51.2,102.4,200],
                  'subsample':     uniform(0.6, 0.4)}

In [206]:
param_grid_xgbr_high_lr = {'gamma':        [0,0.1,0.2,0.4,0.8,1.6,3.2],
                  'learning_rate': [0.300000012, 0.4, 0.5, 0.6, 0.7],
                  'max_depth':     [5,6,7,8,9,10],
                  'n_estimators':  [50,65,80,100,115,130,150],
                  'reg_alpha':     [51.2,102.4,200],
                  'reg_lambda':    [0,0.1,0.2,0.4,0.8,1.6,3.2],
                  'subsample':     uniform(0.6, 0.4)}

In [196]:
param_grid_xgbr = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
                  'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.300000012, 0.4, 0.5, 0.6, 0.7],
                  'max_depth': [5,6,7,8,9,10,11,12,13,14],
                  'n_estimators': [50,65,80,100,115,130,150],
                  'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
                  'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
                  'subsample': uniform(0.6, 0.4)}

In [197]:
from sklearn.model_selection import RandomizedSearchCV

In [207]:
xgbr_cv = RandomizedSearchCV(XGBRegressor(),
                            param_distributions = param_grid_xgbr_high_lr,
                            n_jobs = -1,
                            scoring = 'neg_mean_absolute_error')

In [208]:
xgbr_cv_model = xgbr_cv.fit(X_train_s, y_train)

In [209]:
xgbr_pred_test = xgbr_cv_model.predict(X_test_s)
xgbr_pred_test_inv = inv_boxcox(xgbr_pred_test, lam)

In [210]:
scores(y_test_inv, xgbr_pred_test_inv)

Mean Absolute Error 24519.84082648036
Root Mean Squared Error 64019.40416984486
Mean Squared Log Error 1.1146989141029404
R2 Score -0.1348846714112315


In [222]:
xgbr_cv_low_lr = RandomizedSearchCV(XGBRegressor(),
                            param_distributions = param_grid_xgbr_low_lr,
                            n_jobs = -1,
                            scoring = 'neg_mean_squared_error',
                            n_iter = 30)

In [ ]:
xgbr_cv_model_low_lr = xgbr_cv_low_lr.fit(X_train_s, y_train)

In [220]:
xgbr_pred_test_low_lr = xgbr_cv_model_low_lr.predict(X_test_s)
xgbr_pred_test__low_lr_inv = inv_boxcox(xgbr_pred_test_low_lr, lam)

In [221]:
scores(y_test_inv, xgbr_pred_test__low_lr_inv )

Mean Absolute Error 23892.761940789558
Root Mean Squared Error 62820.80250537486
Mean Squared Log Error 0.9216315511828994
R2 Score -0.09278678500312787


# Todo:
* Add dimensionality reduction
* Make a better model